In [1]:
import optuna
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import KFold 
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error

warnings.filterwarnings('ignore')
seed = 27

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

test_ids = test['id']

In [3]:
train.describe()

,id,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
count,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000
mean,45307.000000,0.517098,0.401679,0.135464,0.789035,0.340778,0.169422,0.225898,9.696794
std,26158.441658,0.118217,0.098026,0.038008,0.457671,0.204428,0.100909,0.130203,3.176221
min,0.000000,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,22653.500000,0.445000,0.345000,0.110000,0.419000,0.177500,0.086500,0.120000,8.000000
50%,45307.000000,0.545000,0.425000,0.140000,0.799500,0.330000,0.166000,0.225000,9.000000
75%,67960.500000,0.600000,0.470000,0.160000,1.067500,0.463000,0.232500,0.305000,11.000000
max,90614.000000,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [4]:
train.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [5]:
train['Sex'] = train['Sex'].map({'M': 0, 'F': 1, 'I': 2})
test['Sex'] = test['Sex'].map({'M': 0, 'F': 1, 'I': 2})

features_to_scale = ['Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1', 'Whole weight.2', 'Shell weight']

scaler = StandardScaler()
train[features_to_scale] = scaler.fit_transform(train[features_to_scale])
test[features_to_scale] = scaler.transform(test[features_to_scale])

In [6]:
X = train.drop(['id', 'Rings'], axis=1)
y = train['Rings']

X_test = test.drop('id', axis=1)

In [7]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 100.0, log=True)      
    }

    cv = KFold(n_splits=10, shuffle=True, random_state=seed)

    scores = []
    for train_idx, valid_idx in cv.split(X, y):
        X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

        model = CatBoostRegressor(**params, random_state=seed)
        model.fit(X_train_fold, y_train_fold, verbose=False)

        y_pred = model.predict(X_valid_fold)
        score = np.sqrt(mean_squared_log_error(y_valid_fold, y_pred))

        scores.append(score)

    return np.mean(scores)

In [8]:
study = optuna.create_study(direction='minimize', study_name="CatBoostRegressor")
study.optimize(objective, n_trials=250, n_jobs=-1, catch=(RuntimeError,))

[I 2024-04-01 10:42:03,026] A new study created in memory with name: CatBoostRegressor
[W 2024-04-01 10:42:40,475] Trial 0 failed with parameters: {'iterations': 707, 'learning_rate': 0.026081718167402266, 'colsample_bylevel': 0.050777087434467544, 'min_data_in_leaf': 20, 'depth': 4, 'l2_leaf_reg': 0.13010857584848395} because of the following error: RuntimeError('Attempt to pop from an empty stack').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_18/3472799885.py", line 19, in objective
    model.fit(X_train_fold, y_train_fold, verbose=False)
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 5807, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 2374, in _fit
   

In [9]:
best_params = study.best_params

model = CatBoostRegressor(**best_params, random_state=seed)
model.fit(X, y, verbose=False)

preds = model.predict(X_test)

In [10]:
submission = pd.DataFrame({'id': test_ids, 'Rings': preds})
submission.to_csv(f'CatBoostRegressor_{study.best_value:.6f}.csv', index=False)
submission.head()

,id,Rings
0,90615,9.775336
1,90616,9.711549
2,90617,10.213126
3,90618,10.154526
4,90619,7.600985
